In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
import requests
from tqdm import tqdm
import time
import ast
import astunparse
from rank_bm25 import BM25Okapi
import pandas as pd

# GitHub API 인증 토큰 (반드시 실제 토큰으로 변경)
github_token = "use your own token"
if github_token is None:
    raise ValueError("GitHub API token not set. Please set the GITHUB_TOKEN environment variable.")

# CodeT5 모델 및 토크나이저 로드
model_name = "Salesforce/codet5-small"                      # Salesforce/codet5-base 혹은 codet5-large 같은 더 큰 모델 사용 가능능
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

In [ ]:
################################################################
# 1. 데이터셋 로드
################################################################
# train 용 데이터셋 로드
dataset = load_dataset("princeton-nlp/SWE-bench_Lite", split="dev")
# evaluation 용 데이터셋 로드드
eval_dataset = load_dataset("princeton-nlp/SWE-bench_Lite", split="dev")

In [ ]:
################################################################
# 2. GitHub API 관련 함수
################################################################
# GitHub API를 사용하여 변경된 파일 목록 가져오는 함수
def get_changed_files_api(repo_owner, repo_name, base_commit, github_token):
    api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/compare/{base_commit}^...{base_commit}"
    headers = {
        "Authorization": f"token {github_token}",
        "Accept": "application/vnd.github.v3+json"
    }
    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        comparison = response.json()
        changed_files = [file["filename"] for file in comparison["files"]]
        return changed_files
    except requests.exceptions.RequestException as e:
        print(f"Error getting changed files for commit {base_commit}: {e}")
        return None

# GitHub API를 사용하여 변경된 파일들의 코드를 가져오는 함수
def get_code_from_commit_api(repo_owner, repo_name, commit_hash, changed_files, github_token):
    headers = {
        "Authorization": f"token {github_token}",
        "Accept": "application/vnd.github.v3.raw"
    }
    base_code_dict = {}
    for file_path in changed_files:
        api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{file_path}?ref={commit_hash}"
        try:
            response = requests.get(api_url, headers=headers)
            response.raise_for_status()
            base_code_dict[file_path] = response.text
        except requests.exceptions.RequestException as e:
            print(f"Error retrieving code for file {file_path} in commit {commit_hash}: {e}")
            return None
    return base_code_dict

In [ ]:
################################################################
# 3. AST 기반 코드 파싱
# 주석이나 불필요한 부분을 미리 제거하여 파싱 속도를 높일 수 있음
################################################################
# FunctionVisitor 클래스
# 주어진 코드에서 동기/비동기 함수 정의를 찾아내어, 함수 이름, 코드, 시작/끝 라인, 그리고 파일 경로 등의 정보를 저장
class FunctionVisitor(ast.NodeVisitor):
    def __init__(self, file_path):
        super().__init__()
        self.functions = []
        self.file_path = file_path

    def visit_FunctionDef(self, node):
        self.functions.append({
            "name": node.name,
            "code": astunparse.unparse(node),
            "start_line": node.lineno,
            "end_line": node.end_lineno,
            "filepath": self.file_path
        })
        self.generic_visit(node)

    def visit_AsyncFunctionDef(self, node):
        self.functions.append({
            "name": node.name,
            "code": astunparse.unparse(node),
            "start_line": node.lineno,
            "end_line": node.end_lineno,
            "filepath": self.file_path
        })
        self.generic_visit(node)

# AST 파싱 함수
# 코드 텍스트를 파싱하여 AST를 생성하고, 위의 FunctionVisitor를 사용해 함수 단위로 코드를 분할
# 파싱 오류가 발생하면 오류 메시지를 출력하고 빈 리스트를 반환
def split_code_into_functions_ast(code_text, file_path):
    try:
        tree = ast.parse(code_text)
        visitor = FunctionVisitor(file_path)
        visitor.visit(tree)
        return visitor.functions
    except (SyntaxError, ValueError) as e:
        print(f"Syntax/Value error in code: {file_path}\n{e}")
        return []

In [ ]:
################################################################
# 4. 패딩 함수
################################################################
# 모델 입력 및 어텐션 마스크의 길이를 512로 맞추기 위해 기본 패딩 토큰과 0을 반환하는 함수
def pad_int_list(pad_token_id=1, length=512):
    return [pad_token_id]*length

def pad_attention_list(length=512):
    return [0]*length

In [ ]:
################################################################
# 5. 데이터 전처리 함수
################################################################
# 데이터 전처리 함수 (BM25, AST 사용)
'''
1. 정보 추출: repo, base_commit, 문제 설명, 그리고 패치 정보를 추출
2. GitHub API 호출: 변경된 파일 목록과 각 파일의 코드를 가져옴
3. AST 파싱: 코드에서 함수 단위로 분할
4. BM25 검색: 문제 설명과 가장 관련성 높은 함수 3개 선택 -> 상위 3개 함수 대신, 더 많은 후보를 선택한 후, 후처리나 re-ranking 단계를 거치는 방법도 고려
5. Prompt 구성: "Fix the bug:"와 선택된 코드 정보를 포함하는 입력 텍스트 생성
6. 토큰화 및 패딩: 입력 및 label(패치) 데이터를 512 토큰 길이로 맞춤
'''
def preprocess_function(batch):
    out_input_ids = []
    out_attention_mask = []
    out_labels = []

    for i in range(len(batch["repo"])):
        try:
            repo = batch["repo"][i]
            base_commit = batch["base_commit"][i]
            problem_statement = batch["problem_statement"][i]
            patch = batch["patch"][i]

            repo_owner, repo_name = repo.split("/")
            changed_files = get_changed_files_api(repo_owner, repo_name, base_commit, github_token)
            if not changed_files:
                print(f"No changed files found for commit {base_commit}. Skipping this example.")
                out_input_ids.append(pad_int_list(tokenizer.pad_token_id, 512))
                out_attention_mask.append(pad_attention_list(512))
                out_labels.append(pad_int_list(tokenizer.pad_token_id, 512))
                continue

            base_code_dict = get_code_from_commit_api(repo_owner, repo_name, base_commit, changed_files, github_token)
            if base_code_dict is None:
                print(f"Failed to retrieve code for commit {base_commit}. Skipping this example.")
                out_input_ids.append(pad_int_list(tokenizer.pad_token_id, 512))
                out_attention_mask.append(pad_attention_list(512))
                out_labels.append(pad_int_list(tokenizer.pad_token_id, 512))
                continue

            all_functions = []
            for file_path, base_code in base_code_dict.items():
                functions = split_code_into_functions_ast(base_code, file_path)
                all_functions.extend(functions)

            if not all_functions:
                print(f"No functions found in commit {base_commit}. Skipping this example.")
                out_input_ids.append(pad_int_list(tokenizer.pad_token_id, 512))
                out_attention_mask.append(pad_attention_list(512))
                out_labels.append(pad_int_list(tokenizer.pad_token_id, 512))
                continue

            tokenized_functions = [tokenizer.tokenize(func["code"]) for func in all_functions]
            bm25 = BM25Okapi(tokenized_functions)
            tokenized_query = tokenizer.tokenize(problem_statement)
            doc_scores = bm25.get_scores(tokenized_query)
            top_n = 3
            top_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:top_n]
            selected_functions = [all_functions[i] for i in top_indices]

            selected_code = ""
            for func_info in selected_functions:
                selected_code += (
                    f"File: {func_info['filepath']}, Function: {func_info['name']}\n"
                    f"Start Line: {func_info['start_line']}, End Line: {func_info['end_line']}\n"
                    f"Code:\n{func_info['code']}\n\n"
                )
    
            input_text = f"Fix the bug: {problem_statement}\nCode:\n{selected_code}"
            input_tokens = tokenizer.encode(input_text)
            if len(input_tokens) > 512:
                input_tokens = input_tokens[:512]
            input_text = tokenizer.decode(input_tokens, skip_special_tokens=True)
    
            model_inputs = tokenizer(input_text, max_length=512, truncation=True)
            labels = tokenizer(patch, max_length=512, truncation=True)
            model_inputs["labels"] = labels["input_ids"]
    
            if "input_ids" not in model_inputs:
                model_inputs["input_ids"] = [tokenizer.pad_token_id] * 512
            if "attention_mask" not in model_inputs:
                model_inputs["attention_mask"] = [0] * 512
            if "labels" not in model_inputs:
                model_inputs["labels"] = [tokenizer.pad_token_id] * 512
    
            input_length = len(model_inputs["input_ids"])
            if input_length < 512:
                model_inputs["input_ids"].extend([tokenizer.pad_token_id] * (512 - input_length))
                model_inputs["attention_mask"].extend([0] * (512 - input_length))
    
            labels_length = len(model_inputs["labels"])
            if labels_length < 512:
                model_inputs["labels"].extend([tokenizer.pad_token_id] * (512 - labels_length))
    
            out_input_ids.append(model_inputs["input_ids"])
            out_attention_mask.append(model_inputs["attention_mask"])
            out_labels.append(model_inputs["labels"])
    
        except Exception as e:
            print(f"[Error] {e}")
            out_input_ids.append([])
            out_attention_mask.append([])
            out_labels.append([])
    
    return {
        "input_ids": out_input_ids,
        "attention_mask": out_attention_mask,
        "labels": out_labels
    }

# convert_to_features 함수
# map 함수에서 호출하여 각 배치를 전처리 함수에 전달
def convert_to_features(batch):
    return preprocess_function(batch)


In [ ]:
################################################################
# 6. 데이터셋 전처리 및 필터링
################################################################
# 원본 데이터셋 전처리 및 필터링
# 원본 데이터셋에 대해 전처리 함수를 적용한 후, 전처리 결과가 빈 값이 아닌 것만 남김
# 전처리 후 사용되는 열은 "input_ids", "attention_mask", "labels"
mapped_dataset = dataset.map(
    convert_to_features,
    batched=True,
    batch_size=1,
    load_from_cache_file=False,
    keep_in_memory=True
)

print("After map =>", mapped_dataset.column_names)

def nonempty_filter(example):
    return len(example["input_ids"]) > 0

mapped_dataset = mapped_dataset.filter(nonempty_filter)
print("After filter =>", mapped_dataset.column_names)

# 평가 데이터셋 전처리 및 필터링
mapped_eval_dataset = eval_dataset.map(
    convert_to_features,
    batched=True,
    batch_size=1,
    load_from_cache_file=False
)
mapped_eval_dataset = mapped_eval_dataset.remove_columns(
    [col for col in mapped_eval_dataset.column_names if col not in ["input_ids", "attention_mask", "labels"]]
)
mapped_eval_dataset = mapped_eval_dataset.filter(nonempty_filter)

In [ ]:
################################################################
# 7. 모델 학습
################################################################
training_args = Seq2SeqTrainingArguments(
    output_dir="./codet5-finetuned-swe-bench-optimized",
    per_device_train_batch_size=2,  # 증가 가능
    per_device_eval_batch_size=2,   # 증가 가능
    gradient_accumulation_steps=2,  # 충분한 메모리로 accumulation 줄일 수 있음
    learning_rate=3e-5,             # 실험에 따라 조정 (큰 배치 사이즈는 학습률이 높아져야 함)
    num_train_epochs=100,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    dataloader_num_workers=2,       # 더 많은 워커 사용 가능
    remove_unused_columns=True,
    report_to="tensorboard",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=mapped_dataset,
    eval_dataset=mapped_eval_dataset,
    tokenizer=tokenizer,
)
train_dataloader = trainer.get_train_dataloader()
first_batch = next(iter(train_dataloader))
print("First batch keys:", first_batch.keys())
print("First batch:", first_batch)

trainer.train()
trainer.save_model()

In [ ]:
################################################################
# 8. 추론(패치 생성)
################################################################
# 추론을 위한 준비 (파인튜닝된 모델 로드)
model = AutoModelForSeq2SeqLM.from_pretrained("./codet5-finetuned-swe-bench-optimized")
model.to(device)

# 추론 데이터셋 로드 및 패치 생성 함수
'''
1. 입력 정보 추출: repo, commit, 문제 설명을 가져옴
2. GitHub API 호출: 변경된 파일과 코드를 가져옴
3. AST & BM25: 함수 단위로 분할 후 문제 설명과 가장 관련있는 상위 3개 함수를 선택
4. Prompt 구성: 선택된 함수 정보와 문제 설명을 포함하는 텍스트를 생성
5. 모델 추론: 이 prompt를 토큰화하여 모델에 입력하고, generate() 함수로 패치를 생성
'''
output_dir = "codet5_patches_bm25"
os.makedirs(output_dir, exist_ok=True)

test_dataset = load_dataset("princeton-nlp/SWE-bench_Lite", split="test")

def generate_patch(example):
    repo_owner = example["repo"].split("/")[0]
    repo_name = example["repo"].split("/")[1]
    commit_hash = example["base_commit"]
    problem_statement = example["problem_statement"]

    changed_files = get_changed_files_api(repo_owner, repo_name, commit_hash, github_token)
    if not changed_files:
        print(f"No changed files found for commit {commit_hash}. Skipping.")
        return None

    changed_files = [f for f in changed_files if f.endswith(".py")]
    if not changed_files:
        print(f"No .py files found for commit {commit_hash}. Skipping.")
        return None

    base_code_dict = get_code_from_commit_api(repo_owner, repo_name, commit_hash, changed_files, github_token)
    if base_code_dict is None:
        print(f"Failed to retrieve code for commit {commit_hash}. Skipping.")
        return None

    all_functions = []
    for file_path, base_code in base_code_dict.items():
        funcs = split_code_into_functions_ast(base_code, file_path)
        all_functions.extend(funcs)
    if not all_functions:
        print(f"No functions found in commit {commit_hash}. Skipping.")
        return None

    tokenized_functions = [tokenizer.tokenize(func["code"]) for func in all_functions]
    bm25 = BM25Okapi(tokenized_functions)
    tokenized_query = tokenizer.tokenize(problem_statement)
    doc_scores = bm25.get_scores(tokenized_query)
    top_n = 3
    top_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:top_n]
    selected_functions = [all_functions[i] for i in top_indices]

    selected_code = ""
    for func_info in selected_functions:
        selected_code += (
            f"File: {func_info['filepath']}, Function: {func_info['name']}\n"
            f"Start Line: {func_info['start_line']}, End Line: {func_info['end_line']}\n"
            f"Code:\n{func_info['code']}\n\n"
        )

    prompt_input = (
        f"Issue: {problem_statement}\n"
        f"Code Context:\n{selected_code}\n"
        "Task: Provide a patch to fix the issue."
    )
    
    input_tokens = tokenizer.encode(prompt_input)
    if len(input_tokens) > 512:
        input_tokens = input_tokens[:512]
    input_text = tokenizer.decode(input_tokens, skip_special_tokens=True)
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=256, 
            pad_token_id=tokenizer.eos_token_id,
            num_beams=5,
            early_stopping=True
        )
    patch = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return patch

# 추론 루프 및 파일 저장
# 생성된 패치는 codet5_patches_bm25 폴더에 UTF-8 인코딩으로 저장
for idx, example in enumerate(tqdm(test_dataset, desc="Generating Patches")):
    patch = generate_patch(example)
    if patch:
        with open(os.path.join(output_dir, f"patch_{idx}.txt"), "w", encoding="utf-8") as f:
            f.write(patch)
        print(f"--- Example {idx} Patch ---")
        print(patch)
    else:
        print(f"--- Example {idx} Skipped ---")
    torch.cuda.empty_cache()
    time.sleep(1)
